# jc: A Fast and Well-Conditioned Spectral Method

Olver and Townsend [[1]](https://epubs.siam.org/doi/10.1137/120865458) introduce a Petrov-Galerkin method for generating sparse discretizations of linear differential equations.
<div>
<img src="title.png" width="500"/>
</div>

## Motivation
Goal: numerically solve ordinary differential equations and partial differential equations (ODEs/PDEs) very quickly while still maintaining high accuracy

Spectral methods promise high accuracy, but typically produce dense matrices when the continuous DE is discretized. These dense matrices can result in poor runtime scaling with problem size.

Finite difference methods have very sparse matrices and can be very fast. However, they are rather inaccurate.

Olver and Townsend [[1]](https://epubs.siam.org/doi/10.1137/120865458) introduce a technique that combines the speed of sparse finite difference methods with the accuracy of a spectral method.

## Example differential equation
Consider the TDGL equation:
$$\partial_t\psi=\psi+(1+ib)\partial_x^2\psi-(1+ic)|\psi|^2\psi$$
with $\psi=\psi(x,t)$ being the order parameter defined on the interval $x \in [0,L]$. Consider Dirichlet boundary conditions $\psi(0,t)=\psi(L,t) = 0$. Assume $L = 300$, $b = 0.5$, and $c=-1.76$.

This equation is interesting because it has non-linearities, which means we have to play some tricks to apply the methods from the paper.

Goal: write an implicit-explicit Euler method to solve this equation given an initial condition.

## Setup

Import required packages and define our parameters

In [ ]:
using ProgressMeter
using BenchmarkTools
using FFTW
using LinearAlgebra
using SparseArrays
using Plots
using LaTeXStrings

L = 300
b = 0.5
c = -1.76
; # semicolons suppress output in an interactive environment such as a REPL or notebook

## Discretization in time
Let $\psi^n(x) = \psi(x,t_0+n\Delta t)$, where $\Delta t$ is the size of the timestep.
### Implicit/explicit timestepping methods
Consider the general PDE:
$$\partial_t\psi = f(\psi)$$
In the following, we know the value of the unknown $\psi$ at the current timestep $\psi^n = \psi(t_0+n\Delta t)$, and want to determine the value of the unknown at the subsequent timestep $\psi^{n+1} = \psi(t_0 + (n+1)\Delta t)$. We approximate the differential operator $\partial_t$ with the following expression:
$$\partial_t\psi \approx \frac{\psi(t+\Delta t)-\psi(t)}{\Delta t}$$
In the limit as $\Delta t$ goes to zero, this approximation becomes more an more accurate. However, for a finite $\Delta t$, we can consider several cases (e.g. forward-difference, backward-difference, and second-order), each with different properties. The case shown above is equivalent to a backward-Euler method.
#### Explicit (forward-Euler) method:
$$\psi^{n+1} - \psi^n = \Delta tf(\psi^n)$$
#### Implicit (backward-Euler) method:
$$\psi^{n+1} - \psi^n = \Delta tf(\psi^{n+1})$$
#### Implicit (trapezoidal) method:
$$\psi^{n+1} - \psi^n = \frac{1}{2}\Delta t\left[f(\psi^{n+1})+f(\psi^n)\right]$$
The allure of the explicit forward-Euler method is that the solve is trivial, since we don't have to find the roots of $f$. This is not the case for implicit methods. However, if $f$ is linear, i.e. we can write $f(\psi) = \mathcal{L}\psi$ where $\mathcal{L}$ is some linear operator (a matrix or differential/integral operator), it is still quite straightforward to implement. Let's consider the backwards-Euler method:
$$\psi^{n+1}-\psi^n = \Delta t\mathcal{L}\psi^{n+1}$$
Rearranging terms, we have a linear equation for $\psi^{n+1}$ with a right hand side of $\psi^n$:
$$(1-\Delta t\mathcal{L})\psi^{n+1} = \psi^n$$
This can be solved with a direct solver (e.g. LU factorization) and can be quite fast (i.e. linear in the number of elements of $\psi$) depending on the properties of $\mathcal{L}$. The update equation for the trapezoidal method looks very similar:
$$(2 - \Delta t \mathcal{L})\psi^{n+1} = (2 + \Delta t \mathcal{L})\psi^{n+1}$$
If $f$ is nonlinear, however, things get a bit more difficult. One can continue to do a full implicit solve, where one must resort to a nonlinear root-finding method, such as Newton to find the value of $\psi^{n+1}$ for which the update equation is satisfied. However, it's also possible to split $f$ into $f(\psi) = \mathcal{L}\psi + f_{N\mathcal{L}}(\psi)$, where $\mathcal{L}$ is a linear operator and $f_{N\mathcal{L}}$ contains the nonlinear part. This method assumes that the timestep is sufficiently small that $f(\psi^{n+1}) \approx \mathcal{L}\psi^{n+1}+f_{N\mathcal{L}}(\psi^n)$.

Therefore, the implicit-explicit update equation is:
$$(1 - \Delta t\mathcal{L})\psi^{n+1} = \psi^n + \Delta tf_{N\mathcal{L}}(\psi^n)$$
This has the advantage of not requiring a Netwon method, while still retaining some of the nice qualities of an implicit method, like allowing for larger timesteps.

## Discretization in space
Now that we have a linear equation to solve to update our unknown for each timestep $\psi^n(x)$, we need to discretize the spatial operator $\mathcal{L}$.
This is where the key methods from the paper come into play [[1]](https://epubs.siam.org/doi/10.1137/120865458). 

### Chebyshev polynomials, fast transforms, and the Petrov-Galerkin method
The Chebyshev polynomials are related to the Fourier basis functions in that $T_n = \cos(n\Theta)$ where the trigonometric coordinate $\Theta$ is related to the spatial coordinate $x$ by $x = \cos\Theta$.
We can plot the first few Chebyshev polynomials

In [ ]:
plot()
for n=0:3
    plot!(range(-1,1,1000), cos.(n.*acos.(range(-1,1,1000))), label=L"T_%$n(x)")
end
plot!()

We will expand the solution $\psi^{n+1}(x)$ in terms of a trial basis of Chebyshev polynomials:
$$\psi^n(x)\approx\sum_{k=0}^{N-1}\hat{\psi^n_k}T_k(x)$$
We can define our unknown vector in terms of these coefficients $\psi^n=\begin{pmatrix}\hat\psi^n_0&\ldots&\hat\psi^n_{N-1}\end{pmatrix}^T$.

Because of the relationship between Chebyshev polynomials and the Fourier basis (Chebyshev polynomials are just Fourier basis functions in disguise), we can leverage Fast Fourier Transforms to go between $\hat\psi^n_k$ and $\psi^n(x_k)$:

In [ ]:
function chebyshev_nodes(N_grid::Int64)::Vector{Float64}
    x_n = cos.(pi / (N_grid - 1) .* (0:(N_grid-1)))
    reverse!(x_n)
    return x_n
end

function chebyshev_coeffs!(u_n::Vector{<:Number}, pDCT1::FFTW.Plan)
    # pDCT1 = FFTW.plan_r2r(u_n, FFTW.REDFT00, 1)
    # reverse isn't strictly necessary, but it's nicer for plotting
    reverse!(u_n)
    pDCT1 * u_n # in place
    u_n ./= length(u_n) - 1.0
    # c_0, c_(N-1) = 1/2, c_k = 1 otherwise
    u_n[1] /= 2.0
    u_n[end] /= 2.0
    nothing
end

function chebyshev_values!(uhat_k::Vector{<:Number}, pDCT1::FFTW.Plan)
    uhat_k[1] *= 2.0
    uhat_k[end] *= 2.0
    # pDCT1 = FFTW.plan_r2r(uhat_k, FFTW.REDFT00, 1)
    pDCT1 * uhat_k
    uhat_k ./= 2.0
    # reverse isn't strictly necessary, but it's nicer for plotting
    reverse!(uhat_k)
    nothing
end;

Let's use these functions to plot the Chebyshev polynomials again:

In [ ]:
N = 128
x_n = chebyshev_nodes(N)
d = zeros(N)
p_g2c = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p_c2g = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p = plot()
for m = 0:3
    d = zeros(N)
    d[m+1] = 1
    chebyshev_values!(d, p_c2g)
    plot!(p, x_n, d, label=L"T_%$m(x)")
end
xlabel!(p, L"x")
ylabel!(p, L"T_n(x)")
plot(p)

If we plot the error compared to the analytical expression for $T_n(x)$, we can see we're at machine precision:

In [ ]:
p = plot()
for m = 0:3
    d = zeros(N)
    d[m+1] = 1
    chebyshev_values!(d, p_c2g)
    plot!(p, x_n, d.-cos.(m.*acos.(x_n)))
end
xlabel!(p, L"x")
ylabel!(p, "error")
plot(p, legend=false)

After expanding our unknown in terms of Chebyshev polynomials, we are left with a finite number of equations to solve, however they must be translated into a weak form, since we cannot use a computer to enforce our new set of equations to hold for every value of $x$.
In the weak form, we take an inner product over every element of a finite-dimensional test basis and only require that our PDE be satisfied over these inner products. Considering a PDE of the form $\mathcal{L}u = f$ with unknown $u$, forcing term $f$, and differential operator $\mathcal{L}$, this translates into the following set of equations:
$$\begin{aligned}\langle w,\mathcal{L}u\rangle = \langle w,f\rangle \\\forall w \in W_N\end{aligned}$$
where $w$ is any test basis belonging to the finite-dimensional vector space $W_N$.

By selecting ultraspherical/Gegenbauer polynomials for our test basis $W_N$ and Chebyshev polynomials for our trial basis $U_N$, we can ensure that the resulting matrix equation will be sparse.

### Constructing differentiation and conversion operators
Constant coefficient terms in $\mathcal{L}$ aren't too tricky to deal with, because they pop right out of the inner products. How should we deal with differentiation operators though?

Using the inner product: $\langle\partial_xT_n(x),C^{(1)}_{m}(x)\rangle=n\delta_{m,n-1}$ and relationship between successive derivatives of ultraspherical polynomials, we get the following differentiation matrix when operating on our unknown vector of Chebyshev coefficients:
$$D_{\lambda}=2^{\lambda-1}(\lambda-1)!\begin{bmatrix}\overbrace{0 ... 0}^{\lambda~{\rm times}} & 1 &\\&& 2 \\&&& 3\\&&&&\ddots\end{bmatrix}, \lambda>0$$
We can implement this in Julia as so:

In [ ]:
function D(N::Int64, lambda::Int64)::SparseMatrixCSC{Float64,Int}
    return (2^(lambda - 1) * factorial(lambda - 1)) .* spdiagm(lambda => lambda:(N-1))
end;

Conversion between Chebyshev coefficients and the various ultraspherical coefficients can be done through the following matrices
$$\begin{aligned}S_{\lambda}&=\begin{bmatrix}1 & &-\frac{\lambda}{\lambda+2}\\&\frac{\lambda}{\lambda+1}&&-\frac{\lambda}{\lambda+3}\\&&\frac{\lambda}{\lambda+2}&&-\frac{\lambda}{\lambda+4}\\&&&\ddots&&\ddots\end{bmatrix},\lambda>1\\S_0&=\begin{bmatrix}1 & & -\frac{1}{2}\\&\frac{1}{2}&&-\frac{1}{2}\\&&\frac{1}{2}&&-\frac{1}{2}\\&&&\ddots&&\ddots\end{bmatrix}\end{aligned}$$
Which has the following Julia implementation

In [ ]:
function S(N::Int64, lambda::Int64)::SparseMatrixCSC{Float64,Int}
    if lambda >= 1
        return spdiagm(
            0 => lambda ./ (lambda .+ collect(0:(N-1))),
            2 => -lambda ./ (lambda .+ collect(2:(N-1))),
        )
    end
    S0 = spdiagm(0 => ones(N), 2 => -ones(N - 2))
    S0[1, 1] = 2
    return S0 ./ 2
end;

We can now use some of the functions we've created to take some derivatives of basic functions. Take for example the function
$$f(x)=\frac{1}{1+10x^2}$$

In [ ]:
N = 64
x_n = chebyshev_nodes(N)
d = zeros(N)
p_g2c = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p_c2g = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
p = plot()
f = tanh.(x_n*2)
plot!(p,x_n, f, label=L"\tanh(x)")
chebyshev_coeffs!(f, p_g2c)
dfdx = S(N, 0) \ (D(N, 1) * f)
chebyshev_values!(dfdx, p_c2g)
plot!(p,x_n,dfdx, label=L"\frac{d}{dx}\tanh(x)")
xlabel!(p, L"x")
ylabel!(p, L"f")
plot(p)

In [ ]:
plot(x_n,dfdx-2*sech.(2*x_n).^2,label="error")
xlabel!(L"x")

Let's compare with a first-order finite-difference method

In [ ]:
x_n = range(-1,1,N)
f = tanh.(x_n*2)
dfdx = (f[3:end] - f[1:end-2])./(2*(x_n[2] - x_n[1]))
plot(x_n, 2*sech.(2*x_n).^2, label="analytical")
plot!(x_n[2:end-1], dfdx, label="finite difference")
xlabel!(L"x")

In [ ]:
plot(x_n[2:end-1], dfdx - 2*sech.(2*x_n[2:end-1]).^2,label="error")
xlabel!(L"x")

The error for the finite-difference method is nearly on the order of 1%, whereas the error for the spectral method is approaching machine precision.
How different are the runtimes?

In [ ]:
function dfdx_fd(f::Vector{Float64}, dx::Float64)::Vector{Float64}
    return (f[3:end] - f[1:end-2])./(2*dx)
end

function dfdx_us(f::Vector{Float64}, p_g2c::FFTW.Plan, p_c2g::FFTW.Plan, S0::SparseMatrixCSC{Float64,Int}, D1::SparseMatrixCSC{Float64,Int})::Vector{Float64}   
    chebyshev_coeffs!(f, p_g2c)
    dfdx = S0 \ (D1 * f)
    chebyshev_values!(dfdx, p_c2g)
    return dfdx
end

x_n = chebyshev_nodes(N)
d = zeros(N)
p_g2c = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.EXHAUSTIVE)
p_c2g = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.EXHAUSTIVE)
S0 = S(N, 0)
D1 = D(N, 1)

x_n = range(-1,1,N)
dx = x_n[2] - x_n[1]
f = tanh.(x_n*2)
@btime dfdx_fd($f, $dx)
x_n = chebyshev_nodes(N)
f = tanh.(x_n*2)
@btime dfdx_us($f, $p_g2c, $p_c2g, $S0, $D1);

Okay, the finite-difference method is about ten times faster for the same number of points. How good is the accuracy if we increase the number of points for the finite difference method so that the runtime is the same (i.e. for the same runtime, how accurate is the finite difference method compared to the spectral method)?

In [ ]:
N = 1000
x_n = range(-1,1,N)
dx = x_n[2] - x_n[1]

f = tanh.(x_n*2)
@btime dfdx_fd($f, $dx);

In [ ]:
f = tanh.(x_n*2)
plot(x_n[2:end-1], dfdx_fd(f, dx) - 2*sech.(2*x_n[2:end-1]).^2,label="error")
xlabel!(L"x")

Considerably better than with $N = 64$, but a far cry from the accuracy of the spectral method

## Solving TDGL equation in one dimension

Recalling our derived expression for the update equation:
$$(1 - \Delta t\mathcal{L})\psi^{n+1} = \psi^n + \Delta tf_{N\mathcal{L}}(\psi^n)$$

We can plug in the specific operators from the TDGL equation:
$$\partial_t\psi=\psi+(1+ib)\partial_x^2\psi-(1+ic)|\psi|^2\psi$$

First, group the linear and nonlinear terms:
$$\partial_t\psi = \left[1 + (1+ib)\partial_x^2\right]\psi-(1+ic)|\psi|^2\psi$$

Then discretize in time:
$$\left(1 - \Delta t - \Delta t (1+ib)\partial_x^2\right)\psi^{n+1} = \psi^n - \Delta t(1+ic)|\psi^n|^2\psi^n$$

And finally discretize in space:
$$\left((1-\Delta t)S_1S_0 - \Delta t (1+ib)D_2\right)\hat{\psi}^{n+1} = S_1S_0\hat{\psi}^n - \Delta t(1+ic)S_1S_0\mathcal{T}^{-1}(|\psi^n|^2\psi^n)$$

where $\hat{\psi}$ and $\psi$ are the Chebyshev $T$ polynomial coefficients and grid values of the solution $\psi(x)$, $D_2$ is the differentiation matrix, $S_1$ and $S_0$ are conversion matrices between Ultraspherical and Gegenbauer polynomial coefficients, and $\mathcal{T}^{-1}$ is the Chebyshev grid-to-coefficient transform.

$\psi^n$ can be determined from $\hat{\psi}^n$ by using the coefficient-to-grid transform $\mathcal{T}$.


Now, to solve, all we need to do is:
$$\psi^{n+1}\leftarrow \mathbf{L}\textbackslash\mathbf{r}(\psi^{n})$$
Where $\textbackslash$ denotes solving the linear system, $\mathbf{L}$ is the left-hand-side linear operator:
$$\mathbf{L} = (1-\Delta t)S_1S_0 - \Delta t (1+ib)D_2$$
and $\mathbf{r}(\psi^n)$ is the right-hand-side nonlinear operator:
$$\mathbf{r}(\psi^n) = S_1S_0\hat{\psi}^n - \Delta t(1+ic)S_1S_0\mathcal{T}^{-1}(|\psi^n|^2\psi^n)$$

### Boundary conditions and chain rule
Notice that the differentiation matrix of order $\lambda$ has $\lambda$ rows that are zero. This is because the differentiation operator reduces the degree of each polynomial basis by $\lambda$. For a polynomial of degree $\lambda^{\prime}<\lambda$, taking the derivative $\lambda$ times will result in a value of 0.

$$D_{\lambda}=2^{\lambda-1}(\lambda-1)!\begin{bmatrix}\overbrace{0 ... 0}^{\lambda~{\rm times}} & 1 &\\&& 2 \\&&& 3\\&&&&\ddots\end{bmatrix}, \lambda>0$$

This provides a great opportunity to enforce boundary conditions using boundary bordering by replacing the rows in the matrix.

For homogeneous Dirichlet boundary conditions $\psi(0)=\psi(L)=0$, we need two equations. We can enforce these equations by evaluating the solution from its Chebyshev series expansion at the endpoints using the following identity for the Chebyshev polynomials:

$$T_n(\pm 1)=(\pm 1)^n$$

Because the differential operator is defined for coordinates $x \in [-1,1]$ and our interval of interest is on the interval $[0,L]$, we need to rescale the differentiation operator by a factor of $\frac{2}{L}$.

### Defining LHS operator

Let's implement the left-hand-side (LHS) operator:
$$\mathbf{L} = (1-\Delta t)S_1S_0 - \Delta t (1+ib)D_2$$
with boundary bordering (applied to the zero-rows of the $D_2$ differentiation matrix) to enforce the boundary conditions $\psi(0)=\psi(L)=0$

In [ ]:
function GL_LHS(N::Int64, dt::Float64, b::Float64)::AbstractArray{<:Number}
    A = (1-dt).*(S(N,1)*S(N,0)) .- (((2/L)^2)*dt*(1+1im*b)).*D(N,2)
    # add boundary conditions
    A_bc = spzeros(ComplexF64,N,N)
    A_bc[1,:] = (-1).^collect(0:N-1)
    A_bc[2,:] .= 1
    A_bc[3:end,:] = A[1:end-2,:]
    return A_bc
end;

Plotting the entries of our LHS matrix in log-scale, we see it is quite sparse

In [ ]:
dt = 0.02
N = 64
LHS = GL_LHS(N, dt, b)
heatmap(log10.(abs.(Matrix(LHS))), aspect_ratio=:equal, xlim=(0.5,N+0.5),ylim=(0.5,N+0.5), yflip=true)

In [ ]:
LU = lu(GL_LHS(64, dt, b))
spy(abs.(LU.L), title="sparsity of L factor")

In [ ]:
spy(abs.(LU.U), title="sparsity of U factor")

Both of these matrices are very sparse!

In [ ]:
nnz(LU.L)/prod(size(LU.L))

In [ ]:
nnz(LU.U)/prod(size(LU.U))

#### Benchmarking LHS solver with random RHS
To see the power of the sparse matrix, let's see what the runtime scaling is as we refine our grid

In [ ]:
log2N_range = 1:1:16
solve_runtime = Array{Any}(nothing, length(log2N_range))
dt = 0.02
@showprogress for (i,log2N) in enumerate(log2N_range)
    N = 2^log2N
    LHS = GL_LHS(N, dt, b)
    LU = lu(LHS)
    f = rand(ComplexF64,N)
    solve_runtime[i] = @benchmark LU\f
end

In [ ]:
solve_runtime[1]

In [ ]:
solve_runtime[end]

In [ ]:
plot(2 .^ log2N_range, getfield.(median.(solve_runtime), :time)/1e9, xaxis=:log, yaxis=:log, markershape=:circle, label=L"LU\backslash f")
xlabel!("N")
ylabel!("runtime [s]")

### Defining RHS operator

Now that our LHS operator is defined, we're almost done with our solver! We just need to implement the right-hand-side (RHS) operator so that we can update our solution vector for each timestep

In [ ]:
function GL_RHS(
    Ahat::AbstractArray{<:Number},
    S::AbstractArray{<:Number},
    N::Int64,
    dt::Float64,
    c::Float64,
    p_g2c::FFTW.Plan,
    p_c2g::FFTW.Plan,
)::AbstractArray{<:Number}
    chebyshev_values!(Ahat, p_c2g)
    RHS = (1 .- (dt*(1 + 1im*c)).*abs.(Ahat).^2).*Ahat
    chebyshev_coeffs!(RHS, p_g2c)
    RHS = S*RHS
    RHS_bc = zeros(ComplexF64, N)
    RHS_bc[3:end] = RHS[1:end-2]
    return RHS_bc
end;

In [ ]:
rhs_runtime = Array{Any}(nothing, length(log2N_range))
dt = 0.02
@showprogress for (i,log2N) in enumerate(log2N_range)
    N = 2^log2N
    S1S0 = S(N,1)*S(N,0)
    d = zeros(ComplexF64, N)
    p_g2c = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
    p_c2g = FFTW.plan_r2r!(d, FFTW.REDFT00, 1, flags = FFTW.ESTIMATE)
    f = rand(ComplexF64,N)
    rhs_runtime[i] = @benchmark GL_RHS($f,$S1S0,$N,$dt,$c,$p_g2c,$p_c2g)
end

In [ ]:
plot(2 .^ log2N_range, getfield.(median.(solve_runtime), :time)/1e9, xaxis=:log, yaxis=:log, markershape=:circle, label=L"LU\backslash f")
plot!(2 .^ log2N_range, getfield.(median.(rhs_runtime), :time)/1e9, xaxis=:log, yaxis=:log, markershape=:rect, label="eval RHS")
xlabel!("N")
ylabel!("runtime [s]")

### Putting it all together - solving the TDGL equation

In [ ]:
function solve_GL(
    A_0::AbstractArray{<:Number},
    N::Int64,
    dt::Float64,
    b::Float64,
    c::Float64,
    n_iter::Int64,
    d_save::Int64,
)::AbstractArray{<:Number}
    S1S0 = S(N,1)*S(N,0)
    LHS = GL_LHS(N, dt, b)
    LU = lu(LHS)
    A_save = zeros(ComplexF64, N, Int(floor(n_iter/d_save)))
    temp = zeros(ComplexF64, N)
    p_g2c = FFTW.plan_r2r!(temp, FFTW.REDFT00, 1, flags = FFTW.EXHAUSTIVE)
    p_c2g = FFTW.plan_r2r!(temp, FFTW.REDFT00, 1, flags = FFTW.EXHAUSTIVE)
    A = A_0[:]
    chebyshev_coeffs!(A, p_g2c);
    @showprogress for i in 1:n_iter
        A = LU\GL_RHS(A, S1S0, N, dt, c, p_g2c, p_c2g)
        if ((i-1) % d_save) == 0
            temp = A[:]
            chebyshev_values!(temp, p_c2g)
            A_save[:,Int(floor((i-1)/d_save))+1] = temp
        end
    end
    return A_save
end

In [ ]:
N = 512
dt = 0.02
L = 20
b = 0.1
c = -0.1
x_n = L/2 .* (1 .+ chebyshev_nodes(N))
A_0 = sin.((2*pi/L) .* x_n)
N_iter = 10000
d_save = 5
A_solve = solve_GL(A_0, N, dt, b, c, N_iter, d_save);

In [ ]:
N_t = Int(floor(N_iter/d_save))
mag = heatmap(x_n, 1:N_t, abs.(A_solve'), xlabel=L"$x$",ylabel=L"$t/\tau_0$", title=L"$|A|$")
phase = heatmap(x_n, 1:N_t, angle.(A_solve'),xlabel=L"$x$", ylabel=L"$t/\tau_0$", title=L"$\angle A$")
plot(mag, phase, layout = (1,2))
plot!(size=(900,500))

In [ ]:
anim = @animate for i = 1:N_t
    plot(x_n, real.(A_solve[:,i]), imag.(A_solve[:,i]), zcolor=abs.(A_solve[:,i]), marker=:circ, markerstrokewidth=0, linewidth=0)
    plot!(xlim=(0,L), ylim=(-1,1), zlim=(-1,1), clim=(0,1.1), legend=false)
    plot!(proj_type = :ortho, camera = (15, round(atand(1 / √2); digits = 3)))
end

In [ ]:
gif(anim, "anim_tdgl.gif", fps = 25)

## Relationship with generalized TDGL
If we consider for simplicity $\gamma = \mu = 0$ and $\mathbf{A} = 0$, as well as $\epsilon = 1$, then the generalized TDGL equation becomes
$$\partial_t\psi=\psi-|\psi|^2\psi+\partial_x^2\psi$$
which is identical to what we'd written previously with $b = c = 0$.